Notebook which will:
- Retrieve a lightcurve belonging to a ZTF SN.
- Do a catalog search around the coordinates of this SN.

In [ ]:
import requests, os, time

In [ ]:
import ampel_quick_import

In [ ]:
%qi AmpelLogger ZTFIdMapper ZiDataPointShaper T1Document LightCurve T2CatalogMatch 

In [ ]:
snname = "ZTF22aafoqrd"
# Access to the AMPEL data archive assumes an individual *archive token* which can be obtained from 
# https://ampel.zeuthen.desy.de/live/dashboard/tokens
# This currently builds on GitHub authorization for AmpelProject members.
token = os.environ["ARCHIVE_TOKEN"]

In [ ]:
def api_to_lightcurve(name, token, shaper=None):
    """
    Retrieve the alert history of a SN and convert to LightCurve
    
    """
    
    # Setup connection
    endpoint = "https://ampel.zeuthen.desy.de/api/ztf/archive/v3/object/{}/photopoints".format(name)
    header = {"Authorization": "bearer "+token}
    response = requests.get(endpoint, headers=header )
    
    if not response.ok:
        print('... failed to get alert')
        return None
    
    # Convert
    alert = response.json()
    if alert is None:
        print(' ... no alert content')
        return None
    
    pps = [alert['candidate']]
    pps.extend( [prv_cand for prv_cand in alert['prv_candidates'] ] )

    if shaper is None:
        tmplog = AmpelLogger.get_logger()
        shaper = ZiDataPointShaper(logger=tmplog)
    
    stockId = ZTFIdMapper.to_ampel_id(name)
    dps = shaper.process( pps, stockId)
    t1d = T1Document(stock=stockId, link=0)
    return LightCurve.build(t1d, dps)

In [ ]:
lc = api_to_lightcurve(snname, token)

I. Make a catalog match to get a redshift.

In [ ]:
catconfig = {
        'SDSS_spec' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['z', 'bptclass', 'subclass'],
            'all': False,
        },
        'NEDz' : {
            'use' : 'catsHTM',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['ObjType', 'Velocity', 'z'],
        },
        'NEDz_extcats' : {
            'use' : 'extcats',
            'rs_arcsec' : 30.0,
            'keys_to_append' : ['ObjType', 'Velocity', 'z'],
        }, 
        'GLADEv23' : {
            'use' : 'extcats',
            'rs_arcsec' : 10,
            'keys_to_append' : ['z', 'dist', 'dist_err', 'flag1', 'flag2', 'flag3'],
        },
        'LSPhotoZZou' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['photoz','ra','dec','e_photoz','specz','_6','logMassBest','logMassInf','logMassSup'],
            'pre_filter' : None,
            'post_filter' : None,
            'all': False,
        },
        'wiseScosPhotoz' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['zPhoto_Corr','ra','dec','wiseID','w1mCorr','w2mCorr'],
            'pre_filter' : None,
            'post_filter' : None,
        },
        'twoMPZ' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['zPhoto','ra','dec','zSpec'],
            'pre_filter' : None,
            'post_filter' : None,
        },

    }

In [ ]:
# Initalize
logger = AmpelLogger.get_logger()
t2cat = T2CatalogMatch( catalogs = catconfig,
    resource = {"ampel-ztf/catalogmatch":'https://ampel.zeuthen.desy.de/api/catalogmatch/'},
    logger=logger)

In [ ]:
# Get a photopoint from which to get coordinates 
t2cat_out = t2cat.process(lc.photopoints[0])

In [ ]:
t2cat_out